In [3]:
import csv
import math

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import keras
from keras import applications
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import keras.backend as K

from keras.models import Model
from keras.layers import Input, Flatten, Dropout, Concatenate, Activation
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D

from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils.data_utils import get_file

WEIGHTS_PATH = 'https://github.com/wohlert/keras-squeezenet/releases/download/v0.1/squeezenet_weights.h5'

def _fire(x, filters, name="fire"):
    sq_filters, ex1_filters, ex2_filters = filters
    squeeze = Convolution2D(sq_filters, (1, 1), activation='relu', padding='same', name=name + "squeeze1x1")(x)
    expand1 = Convolution2D(ex1_filters, (1, 1), activation='relu', padding='same', name=name + "expand1x1")(squeeze)
    expand2 = Convolution2D(ex2_filters, (3, 3), activation='relu', padding='same', name=name + "expand3x3")(squeeze)
    x = Concatenate(axis=-1, name=name)([expand1, expand2])
    return x

def SqueezeNet(include_top=True, weights=None, input_tensor=None, input_shape=None, pooling=None, classes=1000):

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')
    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = Convolution2D(64, kernel_size=(3, 3), strides=(2, 2), padding="same", activation="relu", name='conv1')(img_input)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool1', padding="valid")(x)

    x = _fire(x, (16, 64, 64), name="fire2")
    y = _fire(x, (16, 64, 64), name="fire3")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)
    
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool3', padding="valid")(x)

    x = _fire(x, (32, 128, 128), name="fire4")
    y = _fire(x, (32, 128, 128), name="fire5")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)   

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='maxpool5', padding="valid")(x)

    x = _fire(x, (48, 192, 192), name="fire6")
    y = _fire(x, (48, 192, 192), name="fire7")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)

    x = _fire(x, (64, 256, 256), name="fire8")
    y = _fire(x, (64, 256, 256), name="fire9")
    
    x = keras.layers.add([x,y])
    x = keras.layers.BatchNormalization()(x)

    if include_top:
        x = Dropout(0.5, name='dropout9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = AveragePooling2D(pool_size=(13, 13), name='avgpool10')(x)
        x = Flatten(name='flatten10')(x)
        x = Activation("softmax", name='softmax')(x)
    else:
        if pooling == "avg":
            x = GlobalAveragePooling2D(name="avgpool10")(x)
        else:
            x = GlobalMaxPooling2D(name="maxpool10")(x)

    model = Model(img_input, x, name="squeezenet")

    if weights == 'imagenet':
        weights_path = get_file('squeezenet_weights.h5',
                                WEIGHTS_PATH,
                                cache_subdir='models')

        model.load_weights(weights_path)

    return model

# read the CSV into memory
prices = []
image_paths = []

data_path = "../datasets/bikes_im/"
with open("../datasets/bikes_classified.csv") as file:
    reader = csv.reader(file)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        name = row[1]
        msrp = row[2]
        label = row[3]
        
        image_path = data_path + index + '.jpg'
        image_paths.append(image_path)
        prices.append(str(label))
        
train_indices = np.load("bikes_train_indices.npy")
test_indices = np.load("bikes_test_indices.npy")
print(train_indices.shape)
print(test_indices.shape)

from imgaug import augmenters as iaa

seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 4)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 4)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)
                # Convert to 1 hot vector
                p = prices[index]
                if p == "25":
                    Y[i,:] = np.array([1,0,0,0])
                if p == "50":
                    Y[i,:] = np.array([0,1,0,0])
                if p == "75":
                    Y[i,:] = np.array([0,0,1,0])
                if p == "100":
                    Y[i,:] = np.array([0,0,0,1])
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
#             X = seq.augment_images(X)
            
            yield (X, Y)

# Hyperparameters

num_settings = 1

hp_dropout = [0.5] * num_settings

#RMSprop
hp_lr = [0.005] * num_settings
hp_rho = [0.9] * num_settings
hp_epsilon = [1e-07] * num_settings
hp_decay = [0.0] * num_settings

# Number of hidden units
hp_hidden = [256] * num_settings

# Minibatch size
hp_mbsize = [64] * num_settings

num_epochs = 1000


Using TensorFlow backend.


(19658,)
(2185,)


In [2]:
# store the results of each setting
train_losses = np.zeros(num_settings)
dev_losses = np.zeros(num_settings)

for setting in range(num_settings):
    model = SqueezeNet(include_top=True)
    
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    model.layers.pop()
    
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Convolution2D(256, (1, 1), padding='valid', name='top_conv', input_shape=(model.layers[-1].output_shape[1:])))
    top_model.add(AveragePooling2D(pool_size=(5, 5), name='top_avgpool'))
    top_model.add(Flatten(input_shape=(model.layers[-1].output_shape[1:]),name='top_flatten'))
    top_model.add(Dropout(hp_dropout[setting], name='top_dropout'))
    top_model.add(Dense(hp_hidden[setting], activation='relu', kernel_initializer='glorot_uniform', name='top_dense'))
    top_model.add(Dense(4, activation='softmax', name='output', kernel_initializer='glorot_uniform'))
    
    # add the model on top of the convolutional base
    new_model = Model(inputs= model.input, outputs = top_model(model.layers[-1].output))
#     new_model = load_model("E:/output/bikes-cnn-PriceNet-Class-Aug/00500.hdf5")
    new_model.summary()
    
    # RMSprop optimizer
#     new_model.compile(loss='mean_squared_error',
#                       optimizer=optimizers.RMSprop(
#                               lr=hp_lr[setting], 
#                               rho=hp_rho[setting], 
#                               epsilon=hp_epsilon[setting], 
#                               decay=hp_decay[setting]))
    new_model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])
    
    
    checkpoint_path = 'E:/output/bikes-cnn-PriceNet-Class-Aug/{epoch:05d}.hdf5'
    
    # keep a checkpoint
    checkpoint = ModelCheckpoint(checkpoint_path, period=5)
    
    
    minibatch_size = hp_mbsize[setting]

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)

    # fine-tune the model
    history = new_model.fit_generator(
        image_generator(test_indices, minibatch_size),
        steps_per_epoch=test_steps,
        epochs=300,
        validation_data=image_generator(test_indices, minibatch_size),
        nb_val_samples=test_steps,
        callbacks=[checkpoint])
    
   
    print("==========")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
maxpool1 (MaxPooling2D)         (None, 55, 55, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
fire2squeeze1x1 (Conv2D)        (None, 55, 55, 16)   1040        maxpool1[0][0]                   
__________________________________________________________________________________________________
fire2expan

C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:56: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Richard\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=35, epochs=300, validation_data=<generator..., callbacks=[<keras.ca..., steps_per_epoch=35)`


Epoch 1/300
35/35 [==============================] - 15s 441ms/step - loss: 1.2915 - acc: 0.3967 - val_loss: 1.0984 - val_acc: 0.4940
Epoch 2/300
35/35 [==============================] - 13s 364ms/step - loss: 1.0916 - acc: 0.5037 - val_loss: 0.9733 - val_acc: 0.5602
Epoch 3/300
35/35 [==============================] - 12s 341ms/step - loss: 0.9699 - acc: 0.5607 - val_loss: 0.8867 - val_acc: 0.6052
Epoch 4/300
35/35 [==============================] - 12s 356ms/step - loss: 0.8782 - acc: 0.6195 - val_loss: 0.7575 - val_acc: 0.7043
Epoch 5/300
35/35 [==============================] - 12s 338ms/step - loss: 0.7778 - acc: 0.6712 - val_loss: 0.7280 - val_acc: 0.6945
Epoch 6/300
35/35 [==============================] - 12s 344ms/step - loss: 0.6754 - acc: 0.7343 - val_loss: 0.7481 - val_acc: 0.6741
Epoch 7/300
35/35 [==============================] - 12s 341ms/step - loss: 0.5916 - acc: 0.7809 - val_loss: 0.5765 - val_acc: 0.7470
Epoch 8/300
35/35 [==============================] - 12s 338ms

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Acc')
plt.ylabel('acc')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [4]:
new_model = load_model('E:/output/bikes-cnn-PriceNet-Class-Aug/00110.hdf5')

In [ ]:
true_label = []
predicted_label = []
for index in test_indices:
    msrp = prices[index]
    true_label.append(str(msrp))
    
    path = image_paths[index]
    img = image.load_img(path, target_size=(224, 224))
    data = np.expand_dims(image.img_to_array(img), axis=0)
    
    # Prediction outputs softmax vector
    prediction = new_model.predict(data)
    
    # Set most confident prediction as label, and convert it to our price scale
    label = np.argmax(prediction) * 25 + 25
    predicted_label.append(str(label))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print("Classification report:\n%s\n"
      % (classification_report(true_label, predicted_label)))
print("Confusion matrix:\n%s" % confusion_matrix(true_label, predicted_label))


In [5]:
new_model.layers[-1].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
top_conv (Conv2D)            (None, 13, 13, 256)       131328    
_________________________________________________________________
top_avgpool (AveragePooling2 (None, 2, 2, 256)         0         
_________________________________________________________________
top_flatten (Flatten)        (None, 1024)              0         
_________________________________________________________________
top_dropout (Dropout)        (None, 1024)              0         
_________________________________________________________________
top_dense (Dense)            (None, 256)               262400    
_________________________________________________________________
output (Dense)               (None, 4)                 1028      
Total params: 394,756
Trainable params: 394,756
Non-trainable params: 0
_________________________________________________________________
